In [1]:
import xgboost as xgb
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, confusion_matrix
from pipelines import preprocessor, preprocessor2

In [7]:
train=pd.read_csv('Datasets/train.csv')
train.shape

<IPython.core.display.Javascript object>

(18506, 21)

In [8]:
X=preprocessor2(train)
X

array([[0., 0., 1., ..., 6., 0., 0.],
       [0., 1., 0., ..., 1., 9., 7.],
       [0., 0., 0., ..., 1., 8., 7.],
       ...,
       [0., 1., 0., ..., 1., 8., 7.],
       [0., 1., 0., ..., 1., 3., 4.],
       [0., 1., 0., ..., 6., 9., 0.]])

In [9]:

X.shape

(18506, 35)

In [23]:
y= train.iloc[:, -1]
y.shape

(18506,)

In [24]:
le=LabelEncoder()
y=le.fit_transform(y)
y

array([0, 0, 1, ..., 1, 1, 3])

In [26]:
list(le.classes_)

['High Cost',
 'Higher Cost',
 'Highest Cost',
 'Low Cost',
 'Lower Cost',
 'Normal Cost']

In [27]:
z=le.inverse_transform(y)
z

array(['High Cost', 'High Cost', 'Higher Cost', ..., 'Higher Cost',
       'Higher Cost', 'Low Cost'], dtype=object)

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, shuffle=True, )

In [ ]:
xgb=XGBClassifier(n_extimators=200.
                 max_depth=8,
                 learning_rate=1.0)
